In [ ]:
from nltk.util import ngrams
from collections import defaultdict
from collections import OrderedDict
from tkinter import *
import string
import time
import gc
from math import log10
import sqlite3
from sqlite3 import Error
start_time = time.time()
import csv
import pandas as pd
print(csv.__file__)
import glob, os
import requests
import os
import re
from bs4 import BeautifulSoup
import sys, time
from itertools import cycle

global keyword
# membuat koneksi
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
    return None
# mengecek kalimat yang tidak ada di korpus kata
def cekWrong(conn,inputWrong):
    x = 0
#   memecah inputan menjadi per kata
    temp_l = inputWrong.split()
    i = 0
    j = 0
#   Membersihkan inputan
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] == 's':
                        j = j + 1
                        continue
                word = word.replace(l," ")
#                     print(j,word[j])
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    token = content.split()
#   cek perkata ada di korpus kata atau tidak
    for teks in token:
        kata.append(teks)
        teksStemming = fetch(teks)
        cur = conn.cursor()
#       query untuk cek yang tidak memiliki tag
        cur.execute('Select kata from korpus_kata where kata = (?) and tag != "Entri tidak ditemukan" and tag != "tidak ditemukan kata yang dicari" and tag != "entri tidak ditemukan" ',(teksStemming,))
        a = cur.fetchone()
        conn.commit()
        if a==None:
#           kaliamt yang tidak ada di korpus kata disimpan di wrong
            wrong.append(x)
        x= x + 1
# fungsi untuk mengambil 3 kata sebelum kalimat singkatan

def kalimatTeks(i):
    n = wrong[i]
    if n >= 3 :
        temp = kata[n-3] + ' ' + kata[n-2]  + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 2 :
        temp = 's' + ' ' + kata[n-2] + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 1 :
        temp = 's' + ' ' + 's' + ' ' + kata[n-1]
        input_teks.append(temp)
    elif n == 0 :
        temp = 's' + ' ' + 's' + ' ' + 's'
        input_teks.append(temp)
    print(n)
# fungsi membersihkan kalimat
def removePunctuations(sen):
    temp_l = sen.split()
    i = 0
    j = 0
#   menghilangkan tanda baca dan membuat jadi huruf kecil
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                if l == "'":
                    if j+1<len(word) and word[j+1] == 's':
                        j = j + 1
                        continue
                word = word.replace(l," ")
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    return content
# memuat korpus kalimat untuk menjadi dataset dan menghitung jumlah dari trigram dan bigram 
def loadCorpus(file_path, bi_dict, tri_dict, vocab_dict):
    w1 = ''    #variabel untuk menyimpan 2 kalimat terakhir untuk token set
    w2 = ''    #variabel untuk menyimpan kalimat terakhir untuk token set
    token = []
    word_len = 0

#   memuat korpus kalimat dan di baca line per line
    with open(file_path,'r', encoding="utf-8") as file:
        print(file)
        for line in file:
            print(line)
#           memecah kalimat menjadi kata
            temp_l = line.split()
            i = 0
            j = 0
#           menghilangkan tanda baca dan mebuat kata menjadi huruf kecil
            for word in temp_l :
                j = 0
                for l in word :
                    if l in string.punctuation:
                        if l == "'":
                            if j+1<len(word) and word[j+1] == 's':
                                j = j + 1
                                continue
                        word = word.replace(l," ")
                    j += 1

                temp_l[i] = word.lower()
                i=i+1   
            content = " ".join(temp_l)
            token = content.split()
#           menghitung jumlah kata
            word_len = word_len + len(token)  
            if not token:
                continue
#           menambahkan kalimat terakhir ke variabel
            if w2!= '':
                token.insert(0,w2)
#           token untuk bigrams
            temp0 = list(ngrams(token,2))
#           menambahkan kalimat dua terakhir ke variabel
            if w1!= '':
                token.insert(0,w1)
#           token untuk trigrams
            temp1 = list(ngrams(token,3))
#           menambahkan kata unik ke vocaulary
            for word in token:
                if word not in vocab_dict:
                    vocab_dict[word] = 1
                else:
                    vocab_dict[word]+= 1
#           menghitung frekuensi dari bigram
            for t in temp0:
                sen = ' '.join(t)
                bi_dict[sen] += 1
#           menghitung frekuensi dari trigram
            for t in temp1:
                sen = ' '.join(t)
                tri_dict[sen] += 1
#           menghitung panjang token
            n = len(token)
#           menambahkan kalimat ke variabel
            if (n -3) >= 0:
                w1 = token[n -3]
            if (n -2) >= 0:
                w2 = token[n -2]
            if (n -1) >= 0:
                w3 = token[n -1]
    return word_len
